In [ ]:
import pandas as pd
from tqdm import tqdm
import json
from google.colab import drive


# ===== File paths =====
folder_path = "/content/drive/MyDrive/phi-3-mini"
files = {
    "Flux-Dev": "meta_captions_Flux-Dev_entities.csv",
    "sd_2": "meta_captions_sd_2_entities.csv",
    "sdxl": "meta_captions_sdxl_entities.csv"
}
baseline_file = "DrawBenchPrompts_entities.csv"

# ===== Load baseline =====
baseline_df = pd.read_csv(f"{folder_path}/{baseline_file}")
baseline_df["Prompts_entities"] = baseline_df["Prompts_entities"].apply(lambda x: eval(x) if pd.notna(x) else [])

# ===== Function to compute ALOHA for one model =====
def compute_aloha(model_file, model_name):
    df = pd.read_csv(f"{folder_path}/{model_file}")
    df["Meta Caption_entities"] = df["Meta Caption_entities"].apply(lambda x: eval(x) if pd.notna(x) else [])

    aloha_scores = []
    for i, row in baseline_df.iterrows():
        baseline_entities = set(row["Prompts_entities"])
        model_entities = set(df.loc[i, "Meta Caption_entities"])

        if len(baseline_entities) == 0:
            score = 1.0 if len(model_entities) == 0 else 0.0
        else:
            correct = baseline_entities & model_entities
            score = len(correct) / len(baseline_entities)
        aloha_scores.append(score)

    avg_aloha = sum(aloha_scores) / len(aloha_scores)
    print(f"{model_name} → Average ALOHA: {avg_aloha:.4f}")
    return avg_aloha

# ===== Compute for all models =====
results = {}
for model_name, model_file in files.items():
    results[model_name] = compute_aloha(model_file, model_name)

# ===== Save to JSON =====
out_path = f"{folder_path}/aloha_scores.json"
with open(out_path, "w") as f:
    json.dump(results, f, indent=4)

print(f" ALOHA scores saved to {out_path}")


Flux-Dev → Average ALOHA: 0.1223
sd_2 → Average ALOHA: 0.0672
sdxl → Average ALOHA: 0.0811
 ALOHA scores saved to /content/drive/MyDrive/phi-3-mini/aloha_scores.json
